<a href="https://colab.research.google.com/github/VijayaBhargavi198/5731Assignments/blob/master/In_class_exercise_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## (1) (10 points) Write a python program to extract the sentiment related terms from the corpus. You may use python package such as polyglot or external lexicon resources in the question. Rank the sentiment related terms by frequency.

In [ ]:
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop = stopwords.words('english')
from nltk.tokenize import word_tokenize 
import re
from textblob import TextBlob

input_df=pd.read_csv("/content/sample_data/Joker_user_reviews.csv")

# cleaning the text data
stop_words=stopwords.words("english")
input_df['review'] = input_df['review'].str.replace(r"\W", " ").str.strip()# 1.To remove special characters and punctuations
input_df['review']= input_df['review'].str.replace(r'\d+',"") #2.To remove Numbers
input_df['cleaned text review'] = input_df['review'].apply(lambda x: " ".join(x.lower() for x in x.split())) #convert the whole text into lower case
input_df['cleaned text review'] = input_df['review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop)) #remove all stop words
input_df['cleaned text review'] = input_df['review'].apply(lambda x: nltk.word_tokenize(x)) #tokenise the cleaned data
# input_df['cleaned text review'] # dataframe column that would be used for further process
#find the word count of all the words from cleaned text reviews
word_count_df = (input_df['cleaned text review']).apply(lambda x: pd.value_counts(x)).sum(axis = 0).reset_index() # counting the term frequency/ word_count
word_count_df.columns = ['sentiment_words', 'word_count']
#find polarity of all the words 
word_polarity=[]
for i in word_count_df['sentiment_words']:
  p=TextBlob(i).sentiment.polarity
  word_polarity.append(p)
word_count_df['polarity']= pd.DataFrame(word_polarity)
# sentiment related words are the words that hold positive or negative value in the sentence, hence words with polarity zero are not needed
sentiment_related_terms = word_count_df.loc[word_count_df['polarity'] != 0]
#ranking them according to the word count
sentiment_related_terms.sort_values(by='word_count', ascending=False).reset_index()# index reset in itself gives the rank

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,index,sentiment_words,word_count,polarity
0,356,good,45.0,0.700
1,235,more,38.0,0.500
2,272,very,32.0,0.200
3,218,many,26.0,0.500
4,43,best,23.0,1.000
...,...,...,...,...
371,1111,psychotic,1.0,-0.500
372,1113,BEST,1.0,1.000
373,1124,frustrated,1.0,-0.700
374,1136,Original,1.0,0.375


## (2) (10 points) Compare the performance of the following tools in sentiment identification: TextBlob (https://textblob.readthedocs.io/en/dev/), VADER (https://github.com/cjhutto/vaderSentiment), TFIDF-based Support Vector Machine (SVM) (Split your data into training and testing data). Take your own annotation as the standard answers.

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize  
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words=stopwords.words("english")
clean_text_review=[]
clean_text=[]
input_df=pd.read_csv("/content/sample_data/Joker_user_reviews.csv")
# cleaning the text data
input_df['review'] = input_df['review'].str.replace(r"\W", " ").str.strip()# 1.To remove special characters and punctuations
input_df['review']= input_df['review'].str.replace(r'\d+',"") #2.To remove Numbers
for a in input_df['review']:
    splitting_words=word_tokenize(str(a))
    for b in splitting_words:
        if b not in stop_words: #3.Removing Stop Words
            b=b.lower() # 4.Coverting text to lower case 
            clean_text.append(b)
    clean_text_review.append(' '.join(clean_text))
    clean_text.clear()
input_df['cleaned text review']=pd.DataFrame(clean_text_review) #column that we would be using for further analysis


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#TEXTBLOB
input_df = input_df.dropna() #there might still be some NAN values, removing those 
input_df['polarity'] = input_df['cleaned text review'].apply(lambda x: TextBlob(x).sentiment.polarity)
def text_sentiment(x):
  if(x > 0):
    return "positive"
  if(x < 0):
    return "negative"
  if(x == 0):
    return "neutral"
input_df['output sentiment'] = input_df['polarity'].apply(lambda x: text_sentiment(x))
input_df
print(input_df[['cleaned text review', 'polarity', 'Sentiment','output sentiment']].head(5))

#find accuracy
from sklearn.metrics import f1_score, accuracy_score
# we have the formula sklearn.metrics.accuracy_score(y_true, y_pred, *, normalize=True, sample_weight=None)
accuracy = accuracy_score(input_df['Sentiment'], input_df['output sentiment'])
textblob_accuracy = accuracy *100
print('The accuracy of the TextBlob sentiment analysis is:', textblob_accuracy)


                                 cleaned text review  ...  output sentiment
0  every movie comes truly makes impact joaquin p...  ...          positive
1  this movie felt alone isolated truly relate yo...  ...           neutral
2  truly masterpiece the best hollywood film one ...  ...          positive
3  joaquin phoenix gives tour de force performanc...  ...          positive
4  most time movies anticipated like end falling ...  ...          positive

[5 rows x 4 columns]
The accuracy of the TextBlob sentiment analysis is: 68.23529411764706


In [ ]:
# VADER
input_df = input_df.dropna()#there might still be some NAN values, removing those
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()
input_df['polarity'] = input_df['cleaned text review'].apply(lambda x:vader.polarity_scores(x)['compound'])
def text_sentiment(x):
  if(x > 0):
    return "positive"
  if(x < 0):
    return "negative"
  if(x == 0):
    return "neutral"
input_df['output sentiment'] = input_df['polarity'].apply(lambda x: text_sentiment(x))
input_df
print(input_df[['cleaned text review', 'polarity', 'Sentiment','output sentiment']].head(5))

#find accuracy
from sklearn.metrics import f1_score, accuracy_score
# we have the formula sklearn.metrics.accuracy_score(y_true, y_pred, *, normalize=True, sample_weight=None)
accuracy = accuracy_score(input_df['Sentiment'], input_df['output sentiment'])
Vader_accuracy = accuracy *100
print( 'The accuracy of the Vedar sentiment analysis is:', Vader_accuracy)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
                                 cleaned text review  polarity output sentiment
0  every movie comes truly makes impact joaquin p...    0.9531         positive
1  this movie felt alone isolated truly relate yo...    0.9648         positive
2  truly masterpiece the best hollywood film one ...    0.9732         positive
3  joaquin phoenix gives tour de force performanc...    0.8316         positive
4  most time movies anticipated like end falling ...   -0.5719         negative
The accuracy of the Vedar sentiment analysis is: 63.52941176470588


In [ ]:
#SVM
input_df = input_df.dropna()#there might still be some NAN values, removing those
import sklearn
from sklearn.model_selection import train_test_split
train, test = sklearn.model_selection.train_test_split(input_df, train_size=0.6, test_size=0.4) #splitting data into test and train
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
#defining the pipeline
pipeline = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', SGDClassifier(max_iter=100, learning_rate='optimal'))])
svm_result = pipeline.fit(train['cleaned text review'], train['Sentiment'])
test['output sentiment'] = svm_result.predict(test['cleaned text review'])
print(test[['Sentiment','output sentiment']].head(10))

#find accuracy
# we have the formula sklearn.metrics.accuracy_score(y_true, y_pred, *, normalize=True, sample_weight=None)
accuracy = accuracy_score(test['Sentiment'], test['output sentiment'])
svm_accuracy = accuracy *100
print('\n', 'The accuracy of the TFIDF-based SVM sentiment model is:', svm_accuracy)



   Sentiment output sentiment
69  negative         positive
37  negative         positive
21  positive         positive
59  positive         positive
46  positive         positive
32  positive         positive
2   positive         positive
45  negative         negative
39  negative         negative
10  positive         positive

 The accuracy of the TFIDF-based SVM sentiment identification is: 76.47058823529412


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


On Running all algorithms, I found the accuracy to be higher in case of SVM compared to textblob and vader, thus i conclude, for my dataset and my modelling svm turned out to be more accurate.
Accuracy of all three models are as follows
Textblob =68.2 
Vader = 63.5 
SVM = 76.47 